In [1]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
import sacrebleu
import torch
import nltk
nltk.download('wordnet')

# Load the saved model and tokenizer
model_checkpoint_path = "en_to_fr_model_checkpoint"  # Path to your saved model
saved_model = MarianMTModel.from_pretrained(model_checkpoint_path).to("cuda")
saved_tokenizer = MarianTokenizer.from_pretrained(model_checkpoint_path)



[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
# Function to load sentences from a file
def load_sentences(file_path, num_sentences=None):
    with open(file_path, "r", encoding="utf-8") as file:
        sentences = [line.strip() for line in file]
    if num_sentences:
        sentences = sentences[:num_sentences]
    return sentences

# Translate a batch of sentences
def translate_batch_batched(sentences, model, tokenizer, batch_size=16, max_length=128):
    model.eval()
    all_predictions = []

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]
        inputs = tokenizer(
            batch_sentences,
            return_tensors="pt",
            max_length=max_length,
            padding=True,
            truncation=True
        )
        input_ids = inputs["input_ids"].to("cuda")
        attention_mask = inputs["attention_mask"].to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length
            )
        batch_predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        all_predictions.extend(batch_predictions)

    return all_predictions


In [4]:
# Evaluate BLEU, METEOR, and TER scores
def evaluate_model(predictions, targets):
    # Tokenize the references and predictions for METEOR
    tokenized_references = [[ref.split()] for ref in targets]  # Nested list for BLEU
    tokenized_hypotheses = [hyp.split() for hyp in predictions]

    # BLEU Score
    bleu_score = corpus_bleu(tokenized_references, tokenized_hypotheses)

    # METEOR Score
    meteor_scores = [
        meteor_score([ref.split()], hyp.split())
        for ref, hyp in zip(targets, predictions)
    ]
    avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

    # TER Score
    ter = sacrebleu.corpus_ter(predictions, [targets])

    print(f"BLEU Score: {bleu_score * 100:.2f}")
    print(f"METEOR Score: {avg_meteor_score:.2f}")
    print(f"TER Score: {ter.score:.2f}")

    return bleu_score, avg_meteor_score, ter.score

In [6]:
# Main script
source_file = "europarl-v7.fr-en.en"  # File containing source sentences
target_file = "europarl-v7.fr-en.fr europarl-v7.fr-en.en"  # File containing target sentences
predictions_file = "predictions.fr"  # File to save predictions
num_sentences = 1500  # Limit to first 1500 sentences

# Load source and target sentences
source_sentences = load_sentences(source_file, num_sentences=num_sentences)
target_sentences = load_sentences(target_file, num_sentences=num_sentences)

# Translate source sentences
predictions = translate_batch_batched(source_sentences, saved_model, saved_tokenizer)

# Save predictions to file
with open(predictions_file, "w", encoding="utf-8") as file:
    for prediction in predictions:
        file.write(prediction + "\n")
print(f"Predictions saved to '{predictions_file}'.")

Predictions saved to 'predictions.fr'.


In [7]:
bleu, meteor, ter = evaluate_model(predictions, target_sentences)

BLEU Score: 13.89
METEOR Score: 0.32
TER Score: 77.75
